In [238]:
import inspect, os
from getpass import getpass
import openai
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS, Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import HumanMessage, AIMessage

from langchain.prompts import PromptTemplate
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import AzureOpenAI
from dotenv import load_dotenv
from langchain.chains import LLMChain, ConversationChain,ConversationalRetrievalChain
from langchain.chains.conversation.memory import (ConversationBufferMemory,
                                                  ConversationSummaryMemory,
                                                  ConversationBufferWindowMemory,
                                                  ConversationKGMemory)
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.callbacks import get_openai_callback
import tiktoken
from langchain import PromptTemplate

Przygotowanie bazy danych

In [239]:
files_path = './data/'
loaders = [PyPDFLoader(os.path.join(files_path,fn)) for fn in os.listdir(files_path)]
index = VectorstoreIndexCreator(
        vectorstore_cls=Chroma,
        embedding=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2'),
        text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
).from_loaders(loaders)

Inicjalizacja modelu

In [240]:
load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY") 
openai.api_type = "azure"
openai.api_base = "https://open-ai-maspex.openai.azure.com"
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [241]:
model = AzureOpenAI(
        deployment_name="davinci",
        model_name='text-davinci-003',
        temperature=0.2,
        max_tokens=100,
        top_p=0.9,
        frequency_penalty = 0.2,
        presence_penalty = 0.2,
        stop=["."]
    )

WARNING! stop is not default parameter.
                    stop was transferred to model_kwargs.
                    Please confirm that stop is what you intended.


Prompt plus zliczanie tokenów

In [242]:
def count_tokens(chain, question):
    with  get_openai_callback() as cb:
        res = chain.run(question)
        print(f'You spent a total of {cb.total_tokens}')
    return res

In [255]:
memory = ConversationBufferWindowMemory(k = 2, memory_key='chat_history', return_messages=True, output_key='answer')


In [257]:
retriever = index.vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3, "include_metadata": True})

In [258]:
from langchain.prompts import ChatPromptTemplate
template = """
Use mainly the following pieces of context to answer the question at the end. If you don't understand part of question, rely on your general knowlegde. Answer truthfully.
Answer the question shortly in Polish. \n\n{context} \n\nQuestion: {question} \nHelpful Answer:"""

prompt = PromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages

chain = ConversationalRetrievalChain.from_llm(
        llm = model,
        memory=memory,
        retriever=retriever,
        # return_source_documents=True,
        get_chat_history=lambda h : h,
        verbose=False)


chain.combine_docs_chain.llm_chain.prompt.template = template



In [259]:
response = chain({"question": "Czy podczas pracy zdalnej mogę pracować na własnym sprzęcie?"})
print(f"{response['answer']}\n")


 Tak, ale musisz spełnić określone wymagania techniczne i zgłosić to do działu IT



In [260]:
response = chain({"question": "A na sprzęcie posiadającym nielegalne oprogramowanie?"})
print(f"{response['answer']}\n")

 Niedozwolone jest podejmowanie pracy zdalnej na sprzęcie posiadającym nielegalne oprogramowanie



In [266]:
print(response['chat_history'])

[HumanMessage(content='Mogę pracować zdalnie w McDonaldzie?', additional_kwargs={}, example=False), AIMessage(content=' Nie, praca zdalna w McDonaldzie nie jest dozwolona', additional_kwargs={}, example=False), HumanMessage(content='Dlaczego?', additional_kwargs={}, example=False), AIMessage(content=' Praca zdalna w McDonaldzie nie jest dozwolona, ponieważ niedozwolone jest podejmowanie pracy zdalnej w miejscach publicznych, takich jak kawiarnie, restauracje, galerie handlowe', additional_kwargs={}, example=False)]


In [262]:
response = chain({"question": "Mogę pracować zdalnie w McDonaldzie?"})
print(f"{response['answer']}\n")

 Nie, praca zdalna w McDonaldzie nie jest dozwolona



In [263]:
response = chain({"question": "Dlaczego?"})
print(f"{response['answer']}\n")

 Praca zdalna w McDonaldzie nie jest dozwolona, ponieważ niedozwolone jest podejmowanie pracy zdalnej w miejscach publicznych, takich jak kawiarnie, restauracje, galerie handlowe



In [265]:
response = chain({"question": "Jaka jest bezpieczna odległość w firmie?"})
print(f"{response['answer']}\n")

 Pracownicy i goście firmy McDonald's powinni utrzymywać bezpieczną odległość co najmniej 1,5 m



GPT 3.5

In [321]:
retriever = index.vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4, "include_metadata": True})

In [328]:
model = AzureOpenAI(
        deployment_name="chat",
        model_name='gpt-35-turbo',
        temperature=0.1,
        max_tokens=100,
        top_p=0.1,
        frequency_penalty = 0.1,
        presence_penalty = 0.1,
        stop=["."]
    )

WARNING! stop is not default parameter.
                    stop was transferred to model_kwargs.
                    Please confirm that stop is what you intended.


In [329]:
chain = ConversationalRetrievalChain.from_llm(
        llm = model,
        memory=memory,
        retriever=retriever,
        # return_source_documents=True,
        get_chat_history=lambda h : h,
        verbose=False)


chain.combine_docs_chain.llm_chain.prompt.template = template



In [330]:
response = chain({"question": "Czy podczas pracy zdalnej mogę pracować na własnym sprzęcie?"})
print(f"{response['answer']}\n")

 "Czy mogę pracować na własnym sprzęcie podczas pracy zdalnej?" means "Can I work on my own equipment during remote work?"

A: Czy mogę pracować na własnym sprzęcie podczas pracy zdalnej?
B: Tak, ale musisz spełnić wymagania techniczne



In [331]:
response = chain({"question": "A na sprzęcie posiadającym nielegalne oprogramowanie?"})
print(f"{response['answer']}\n")

 "Czy mogę pracować na własnym sprzęcie podczas pracy zdalnej?" means "Can I work on my own equipment during remote work?"

Conversation:
A: Czy mogę pracować na własnym sprzęcie podczas pracy zdalnej?
B: Nie, musisz korzystać z urządzeń udostępnionych przez pracodawcę



In [303]:
print(response['chat_history'])

[HumanMessage(content='Jaka jest bezpieczna odległość w firmie?', additional_kwargs={}, example=False), AIMessage(content=' "I\'m sorry, I didn\'t understand the question', additional_kwargs={}, example=False), HumanMessage(content='Czy podczas pracy zdalnej mogę pracować na własnym sprzęcie?', additional_kwargs={}, example=False), AIMessage(content=" Tak, możesz pracować na swoim sprzęcie, ale musisz spełnić wymagania bezpieczeństwa IT', additional_kwargs={}, example=False)]<|im_sep|>", additional_kwargs={}, example=False)]


In [295]:
response = chain({"question": "Mogę pracować zdalnie w McDonaldzie?"})
print(f"{response['answer']}\n")

  Nie, nie możesz pracować zdalnie w McDonaldzie na swoim sprzęcie posiadającym nielegalne oprogramowanie



In [296]:
response = chain({"question": "Dlaczego?"})
print(f"{response['answer']}\n")

  Nie możesz pracować zdalnie w miejscach publicznych, takich jak kawiarnie, restauracje, galerie handlowe, gdzie osoby postronne mogłyby usłyszeć fragmenty służbowych rozmów, przeczytać dokumenty służbowe, w tym również korespondencję mailową lub zapoznać się innymi z fragmentami (obszarami) pracy wy



In [298]:
response = chain({"question": "Jaka jest bezpieczna odległość w firmie?"})
print(f"{response['answer']}\n")

 "I'm sorry, I didn't understand the question

